In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os, sys
sys.path.append('/home/hrai/codes/hpe_library')
from lib_import import *
from my_utils import BatchCamera, Camera
from my_utils import arg_as_list, get_configs
from my_utils import generate_random_segment, generate_random_trajectory, get_aligned_init_torso, get_ap_pose_2d, get_backbone_line_from_torso, get_bounded_segments_idx, get_cam_param, get_input, get_label, get_model_input, get_output, get_pairs, get_part_traj, get_pose_seq_and_cam_param, get_two_point_parts, load_h36m, load_segment_file_from_parameters, make_input, make_one_dimension_list, make_output, split_continuous_indices, MyCustomDataset, parse_args_by_model_name, get_input_gt_for_onevec, get_limb_angle, get_h36m_camera_info, h36m_data_processing
from my_utils import DH_matrix, azim_elev_to_vec, batch_azim_elev_to_vec, batch_inverse_tf, batch_projection, batch_rot_x_matrix, batch_rot_y_matrix, batch_rot_z_matrix, calculate_azimuth_elevation, calculate_rotation_quaternion, draw_arm, draw_subline, draw_torso_direction, frame_vec_to_matrix, generate_batch_tf_from_batch_origin_R, generate_camera_frame, generate_dh_frame, generate_tf_from_origin_R, generate_two_link, generate_vis_frame, generate_world_frame, get_batch_frame_vec_from_keypoints, get_batch_lower_torso_frame_from_keypoints, get_batch_lower_torso_frame_from_pose, get_batch_reference_frame, get_batch_upper_torso_frame_from_keypoints, get_batch_upper_torso_frame_from_pose, get_frame_from_keypoints, get_lower_torso_frame_from_pose, get_reference_frame, get_torso_direction, get_torso_rotation_matrix, get_torso_shape, get_upper_torso_frame_from_pose, inverse_tf, normalize_vector, project_batch_tensor, projection, rotate_torso_by_R, rotate_torso_by_R_for_batch_tensor, rotation_matrix_to_vector_align, rotation_matrix_torso2torso, Appendage, BatchAppendage, BatchDHModel, DHModel, calculate_batch_azimuth_elevation
from my_utils import args_dict_to_namespace, construct_torso_from_output, denormalize_motionbert_result, get_dataset_info_from_segment_folder, get_inference_from_dhdst, get_inference_from_dhdst_torso, get_inference_from_motionbert, get_output_type, get_result, infer_one_segment, load_best_model_for_inference, normalize_input, test_model_by_segment_file
from my_utils import get_logger, log_configs
from my_utils import init_weights, split_array_by_idxs, BaselineModel, Linear, TorsoModel
from my_utils import Camera2ImageCoordinate, MPJPE, MPJPE_for_multiple_pose, MPJPE_for_single_pose, World2CameraCoordinate, World2ImageCoordinate, _sqrt_positive_part, _weak_project, aihub2h36m, array2dict, avgErrorForOneAction, avgErrorForOneActor, avgErrorForOneCamera, camera_to_image_frame, change_bbox_convention, check_max_min, coco2h36m, convert3DResult, draw_skeleton, draw_skeleton_2d, draw_skeleton_both, euclidean_distance, fit3d2h36m, fit3d_load_gt_and_param, get3DResult, getAIHubCameraParameter, getGT, getNumFromImgName, get_batch_h36m_keypoints, get_bbox_area, get_bbox_area_from_pose2d, get_bbox_from_pose2d, get_h36m_keypoint_index, get_h36m_keypoints, get_rootrel_pose, get_video_frame, get_video_info, get_xy_centered_pose, halpe2h36m, infer_box, kookmin2h36m, kookmin2h36m_with_nose, loadAIHubCameraParameter, matrix_to_quaternion, normalize, normalize_array, optimize_scaling_factor, plot_cv2_image, readJSON, readpkl, savepkl, skew_symmetric_matrix, skew_symmetric_matrix_tensor, standardize_quaternion, mpi_inf_3dhp2h36m, get_pose_height, get_batch_bbox_from_pose2d, len_ids, part_ids, h36m_part_keypoints, get_h36m_limb_lens
from my_utils import check_duplicate_training, get_input_output_candidate, get_num_trial, load_args, load_best_model, load_dataset, load_model, prepare_training, run, run_epoch, save_args, split_array_by_idxs
from my_utils import axes_2d, axes_3d, axes_to_compare_pred_gt, draw_3d_pose, draw_bbox, draw_multiple_2d_pose, draw_multiple_3d_pose, draw_one_segment, draw_rotation_matrix, draw_segment, draw_segments, draw_trajectory, get_2d_pose_image, legend_without_duplicate_labels, plot_to_compare_pred_gt, save_h36m_pose_video, show2Dtrajectory, show3Dtrajectory, show_2d_3d, show_whole_segment_trajectories, draw_2d_pose, clear_axes
from my_utils import check_available_frame, check_continuity, draw_base_marker_3d, generate_kookmin_pkl_for_each_video, get_cam_param_kookmin, get_video_frame_kookmin, get_video_num_frame_kookmin, load_pose3d_kookmin, load_csv_kookmin, get_lbot

# 주요 디렉토리 경로
user = getpass.getuser()
alphapose_root = '/home/{}/codes/AlphaPose'.format(user)
motionbert_root = '/home/{}/codes/MotionBERT'.format(user)
kookmin_root = '/home/{}/Datasets/HAAI/국민대데이터/data'.format(user)
ap_kookmin_result_root = alphapose_root + "/examples/kookmin_result_5actions"

assert os.path.isdir(alphapose_root), "AlphaPose root directory is not exist"
assert os.path.isdir(motionbert_root), "MotionBERT root directory is not exist"
#assert os.path.isdir(kookmin_root), "Kookmin root directory is not exist"
#assert os.path.isdir(ap_kookmin_result_root), "AlphaPose Kookmin result directory is not exist"

os.chdir(motionbert_root)
from lib.utils.tools import * # get_config
from lib.utils.learning import * # load_backbone
from lib.utils.args import get_opts_args
from lib.utils.utils_data import flip_data
from lib.model.load_model import load_model
from lib.model.load_dataset import load_dataset
from lib.model.loss import *
from lib.model.training import *
from lib.model.evaluation import *
from lib.data.datareader_kookmin import DataReaderKOOKMIN
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.data.datareader_3dhp import DataReader3DHP
from lib.data.datareader_random_limb import DataReaderRandomLimb
from lib.data.dataset_motion_3d import MotionDataset3D
from lib.model.DHDSTformer import DHDSTformer_total, DHDSTformer_total2, DHDSTformer_total3, DHDSTformer_total4, DHDSTformer_total5, DHDSTformer_total6, DHDSTformer_total7, DHDSTformer_total8, \
    DHDSTformer_limb, DHDSTformer_torso, DHDSTformer_torso2
    
dt_file_mapping = {
    'H36M-GT': 'h36m_gt',
    'H36M-GT-CAM_NO_FACTOR': 'h36m_gt',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST': 'h36m_gt_canonical_3d_same_dist',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST-S15678_TR_54138969_TS_OTHERS': 'h36m_gt_canonical_3d_same_dist_s15678_tr_54138969_ts_others',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_same_dist_tr_s1_ts_s5678',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z': 'h36m_gt_canonical_3d_same_z',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678_BY_CANON1_6_PRED': 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678_BY_CANON2_2_PRED': 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678',
    'H36M-GT-CAM_NO_FACTOR-S15678_TR_54138969_TS_OTHERS': 'h36m_gt_s15678_tr_54138969_ts_others',
    'H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678': 'h36m_gt_tr_s1_ts_s5678',
    'H36M-GT-INPUT_FROM_3D_CANONICAL_SAME_DIST-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_same_dist_tr_s1_ts_s5678',
    'H36M-GT-TR_S1_TS_S5678': 'h36m_gt_tr_s1_ts_s5678',
    'H36M-GT-WORLD_NO_FACTOR': 'h36m_gt',
    'H36M-SH': 'h36m_sh_conf_cam_source_final',
    'H36M_CANONICALIZATION-GT-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678',
    'FIT3D-GT-ALL_TEST': 'fit3d_gt_all_test',
    'FIT3D-GT-CAM_NO_FACTOR-ALL_TEST': 'fit3d_gt_all_test',
    'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST': 'fit3d_gt_canonical_3d_same_z_all_test',
    'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S03': 'fit3d_gt_canonical_3d_same_z_tr_s03',
    'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710': 'fit3d_gt_canonical_3d_same_z_ts_s4710',
    'FIT3D-GT-CAM_NO_FACTOR-TS_S4710': 'fit3d_gt_ts_s4710',
    'FIT3D-GT-TS_S4710': 'fit3d_gt_ts_s4710',
    '3DHP-GT-CAM_NO_FACTOR-POSEAUG_TEST_2929': 'poseaug_3dhp_test',
    '3DHP-GT-CAM_NO_FACTOR-TESTSET': '3dhp_gt_test',
    '3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN': '3dhp_gt_test_all_train',
    '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN': '3dhp_gt_canonical_3d_same_z_test_all_train',
    '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_4': '3dhp_gt_test_TS1_4',
    '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6': '3dhp_gt_test_TS1_6',
    '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6': '3dhp_gt_test_canonical_3d_from_same_z_TS1_6',
     
}

def update_page_properties(page_id, properties):
    url = f"https://api.notion.com/v1/pages/{page_id}"
    data = {
        "properties": properties
    }
    response = requests.patch(url, headers=headers, json=data)
    return response.json()

def update_mpjpe(page_id, mpjpe, p_mpjpe):
    url = f"https://api.notion.com/v1/pages/{page_id}"
    data = {
        "properties": {
            "MPJPE": {
                "number": mpjpe
            },
            "P-MPJPE": {
                "number": p_mpjpe
            }
        },
    }

    response = requests.patch(url, headers=headers, json=data)
    return response.json()

def update_process(page_id, process):
    # process: "To train", "To test", "Complete" 
    url = f"https://api.notion.com/v1/pages/{page_id}"
    data = {
        "properties": {
            "Process": {
                "select": {
                    "name": process
                }
            }
        }
    }
    if process == "Complete": emoji = "✅"
    else: emoji = "🏁"
    data["icon"] = {
        "type": "emoji",
        "emoji": emoji
    }

    response = requests.patch(url, headers=headers, json=data)
    return response.json()

def find_first_uppercase_index(s):
    for index, char in enumerate(s):
        if char.isupper():
            return index
    return -1  # 대문자가 없는 경우 -1 반환

import requests
from tqdm import tqdm
import copy

# Notion API 키 및 데이터베이스 ID 설정
notion = readpkl('notion.pkl')
NOTION_API_KEY = notion['NOTION_API_KEY']
DATABASE_ID = notion['DATABASE_ID']

# 헤더 설정
headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

def get_all_database_pages():
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
    all_pages = []
    has_more = True
    start_cursor = None
    
    while has_more:
        if start_cursor:
            data = {"start_cursor": start_cursor}
        else:
            data = {}
        
        response = requests.post(url, headers=headers, json=data)
        response_data = response.json()
        
        all_pages.extend(response_data["results"])
        has_more = response_data.get("has_more", False)
        start_cursor = response_data.get("next_cursor", None)
    print(f"Total pages: {len(all_pages)}")
    return all_pages

pages = get_all_database_pages()

pages_dict = {}
for page in pages:
    pages_dict[page["id"]] = page
    
model_dict = {}
for page in pages:
    page_id = page["id"]
    property = page["properties"]
    try:
        model_name = property["Model"]['select']['name']
        page_title = property["Name"]['title'][0]['plain_text']
    except: continue
    #print(model_name, page_title)
    if model_name not in model_dict:
        model_dict[model_name] = {}
    model_dict[model_name][page_title] = page_id


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Total pages: 192


In [67]:
# for item in glob('/home/hrai/codes/MotionBERT/custom_codes/Inference_and_evaluation/experiment_results/*.pkl'):
#     folder_path = os.path.dirname(item)
#     file_name = os.path.basename(item)
#     start_of_dataset = find_first_uppercase_index(file_name[2:])+2-1
#     #print(start_of_dataset, file_name[start_of_dataset], file_name)
#     model_name = file_name[:start_of_dataset]
#     dataset_name = file_name[start_of_dataset+1:].split('.')[0]
#     print(model_name, dataset_name)
#     new_folder = os.path.join(folder_path, model_name)
#     if not os.path.exists(new_folder):
#         os.makedirs(new_folder)
#     new_path = os.path.join(new_folder, dataset_name)
#     print(new_folder)
#     print(new_path)
#     os.rename(item, new_path)

In [7]:
def print_result(model_folder, subset, total_result_dict, action_names, part='whole'):
    #print('Part:', part)
    final_result = []
    final_result_procrustes = []
    summary_table = prettytable.PrettyTable()
    summary_table.field_names = ['test_name'] + action_names # first row
    for action in action_names:
        final_result.append(np.mean(total_result_dict[part]['results'][action]))
        final_result_procrustes.append(np.mean(total_result_dict[part]['results_procrustes'][action]))
    summary_table.add_row(['P1 ({})'.format(part)] + final_result) # second row
    summary_table.add_row(['P2 ({})'.format(part)] + final_result_procrustes) # third row
    
    table_string = summary_table.get_string()
    
    # Total Error
    e1_part = np.mean(np.array(final_result))
    e2_part = np.mean(np.array(final_result_procrustes))
    return e1_part, e2_part, table_string

In [8]:
root = '/home/hrai/codes/MotionBERT/custom_codes/Inference_and_evaluation/experiment_results'
config_root = 'configs/pose3d/'
checkpoint_root = 'checkpoint/pose3d/'

save_summary_txt = False
update_notion_page = True

for model_name in os.listdir(root):
    model_folder = os.path.join(root, model_name)
    save_path = os.path.join(model_folder, 'summary.txt')
    with open(save_path, 'w') as f:
        for dataset_name in os.listdir(model_folder):
            if '.pkl' not in dataset_name: continue
            subset = dataset_name.split('.')[0]
            page_id = model_dict[model_name][subset]
            page_properties = pages_dict[page_id]['properties']
            mpjpe = page_properties['MPJPE']['number']
            p_mpjpe = page_properties['P-MPJPE']['number']
            if mpjpe and p_mpjpe: 
                print(f'[{model_name}] [{subset}] already has MPJPE and P-MPJPE')
                update_process(page_id, "Complete")
                continue
            
            print(model_name, subset)
            dataset_file = os.path.join(model_folder, dataset_name)
            try:
                data = readpkl(dataset_file)
                total_result_dict = data['total_result_dict']
            except Exception as e:
                print(e, model_name, subset, data.keys())
                continue
            
            # init args
            config = model_name + '.yaml'
            input_args = ['--config', config_root+config, '--evaluate', checkpoint_root+model_name+'/best_epoch.bin']
            args, opts = get_opts_args(input_args, verbose=False)
            
            args.subset_list = [subset]
            args.dt_file = dt_file_mapping[args.subset_list[0]]+'.pkl'
            #print(model_name, args.gt_mode, args.subset_list, args.dt_file)
            
            for subset in args.subset_list:
                if 'H36M' in subset: 
                    datareader = DataReaderH36M(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file, input_mode=args.input_mode, gt_mode=args.gt_mode)
                    #action_names = ['Directions', 'Directions 1', 'Discussion 1', 'Discussion 2', 'Eating', 'Eating 1', 'Greeting', 'Greeting 1', 'Greeting 2', 'Phoning', 'Phoning 1', 'Phoning 2', 'Phoning 3', 'Photo', 'Photo 1', 'Posing', 'Posing 1', 'Purchases', 'Purchases 1', 'Sitting', 'Sitting 1', 'SittingDown', 'SittingDown 1', 'Smoking', 'Smoking 1', 'Smoking 2', 'Waiting', 'Waiting 1', 'WalkDog', 'WalkDog 1', 'WalkTogether', 'WalkTogether 1', 'Walking', 'Walking 1']
                elif 'FIT3D' in subset: 
                    datareader = DataReaderFIT3D(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file, input_mode=args.input_mode, gt_mode=args.gt_mode)
                    #action_names = ['band_pull_apart', 'barbell_dead_row', 'barbell_row', 'barbell_shrug', 'burpees', 'clean_and_press', 'deadlift', 'diamond_pushup', 'drag_curl', 'dumbbell_biceps_curls', 'dumbbell_curl_trifecta', 'dumbbell_hammer_curls', 'dumbbell_high_pulls', 'dumbbell_overhead_shoulder_press', 'dumbbell_reverse_lunge', 'dumbbell_scaptions', 'man_maker', 'mule_kick', 'neutral_overhead_shoulder_press', 'one_arm_row', 'overhead_extension_thruster', 'overhead_trap_raises', 'pushup', 'side_lateral_raise', 'squat', 'standing_ab_twists', 'w_raise', 'walk_the_box', 'warmup_1', 'warmup_10', 'warmup_11', 'warmup_12', 'warmup_13', 'warmup_14', 'warmup_15', 'warmup_16', 'warmup_17', 'warmup_18', 'warmup_19', 'warmup_2', 'warmup_3', 'warmup_4', 'warmup_5', 'warmup_6', 'warmup_7', 'warmup_8', 'warmup_9']
                elif '3DHP' in subset:
                    datareader = DataReader3DHP(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file, input_mode=args.input_mode, gt_mode=args.gt_mode)
                    
            action_names = sorted(set(datareader.dt_dataset['test']['action']))
            #print(action_names)
            
            e1_part, e2_part, table_string = print_result(model_folder, subset, total_result_dict, action_names)
            if save_summary_txt:
                f.write(f'subset: {subset}\n')
                f.write(table_string)
                f.write(f'\nProtocol #1 Error (MPJPE): {e1_part} mm\n')
                f.write(f'Protocol #2 Error (P-MPJPE): {e2_part} mm\n')
                f.write('----------------------------------------\n\n')
            if update_notion_page:
                update_mpjpe(page_id, float(f'{e1_part:.2f}'), float(f'{e2_part:.2f}'))
        #break
    #break

MB_train_h36m_gt_with_canonical2_tr_s1_ts_s5678 FIT3D-GT-ALL_TEST already has MPJPE and P-MPJPE


MB_train_h36m_gt_with_canonical2_tr_s1_ts_s5678 H36M-GT already has MPJPE and P-MPJPE
MB_train_h36m_gt_with_canonical2_tr_s1_ts_s5678 H36M-GT-TR_S1_TS_S5678 already has MPJPE and P-MPJPE
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6 already has MPJPE and P-MPJPE
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710 H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678 already has MPJPE and P-MPJPE
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN already has MPJPE and P-MPJPE
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710 FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710 already has MPJPE and P-MPJPE
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710 H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z already has MPJPE

In [39]:
for model_name in model_dict.keys():
    for subset in model_dict[model_name].keys():
        if subset == model_name: continue
        child_id = model_dict[model_name][subset]
        child_page = pages_dict[child_id]
        # train = page['properties']['Train dataset']['select'] 
        # test = page['properties']['Test dataset']['select']
        # if train == None and test == None: continue # parent
        child_properties = child_page['properties']
        subset = child_properties['Name']['title'][0]['text']['content']
        mpjpe = child_properties['MPJPE']['number']
        p_mpjpe = child_properties['P-MPJPE']['number']
        print(subset, mpjpe, p_mpjpe)

H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST None None
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST 162.66 103.25
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678_BY_CANON1_6_PRED 45.94 31.45
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678-BY_CANON2_2_PRED 46.9 31.99
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST-TR_S1_TS_S5678 40.38 27.93
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z None None
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST 126.91 81.25
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678 40.82 27.58
H36M-GT 42.01 32.97
3DHP-GT-POSEAUG_TEST_2929 -1 -1
3DHP-GT-TESTSET -1 -1
FIT3D-GT-ALL_TEST 374.46 190.11
H36M-GT-TR_S1_TS_S5678 42.57 32.6
H36M-GT None None
3DHP-GT-POSEAUG_TEST_2929 -1 -1
3DHP-GT-TESTSET -1 -1
FIT3D-GT-ALL_TEST 167.81 103
H36M-GT-TR_S1_TS_S5678 39.05 31.2
H36M-GT None None
3DHP-GT-POSEAUG_TEST_2929 -1 -1
3DHP-GT-TESTSET -1 -1
H36M-GT-TR_S1_T

In [51]:
model_name = 'MB_train_fit3d_gt_cam_no_factor_tr_s03'
subset = 'H36M-GT-CAM_NO_FACTOR'
page_id = model_dict[model_name][subset]
page = pages_dict[page_id]
properties = page['properties']
mpjpe = properties['MPJPE']['number']
p_mpjpe = properties['P-MPJPE']['number']
print(mpjpe, p_mpjpe)
# properties['MPJPE']['number'] = -1
# properties['P-MPJPE']['number'] = -1
update_mpjpe(page_id, -1, -1)


-1 -1


{'object': 'page',
 'id': '8d83fbe1-2a47-4585-9793-75f4289494e3',
 'created_time': '2024-07-05T01:41:00.000Z',
 'last_edited_time': '2024-07-07T12:10:00.000Z',
 'created_by': {'object': 'user',
  'id': 'f529f665-d931-4f8d-afd7-16643ff86e1d'},
 'last_edited_by': {'object': 'user',
  'id': 'c6d38448-0013-4c0a-91c8-359f6979bf12'},
 'cover': None,
 'icon': {'type': 'emoji', 'emoji': '🏁'},
 'parent': {'type': 'database_id',
  'database_id': '697fd4ef-9dca-42f7-a204-bd1b57e64b83'},
 'archived': False,
 'in_trash': False,
 'properties': {'Input frame': {'id': '%3BvPG',
   'type': 'number',
   'number': 243},
  'Train 2D input': {'id': '%3FY%7BB',
   'type': 'formula',
   'formula': {'type': 'string', 'string': 'GT'}},
  'P-MPJPE': {'id': 'Avig', 'type': 'number', 'number': -1},
  'Parent item': {'id': 'FQBZ',
   'type': 'relation',
   'relation': [{'id': '9b8bd400-75a8-48f7-9716-f35981854864'}],
   'has_more': False},
  'MPJPE': {'id': 'KC%3C%7D', 'type': 'number', 'number': -1},
  'Input sca

In [35]:
properties['Train 2D input']

{'id': '%3FY%7BB',
 'type': 'formula',
 'formula': {'type': 'string', 'string': 'GT'}}